In [0]:
# Nauman Shamim  i19-1610
# Saeed Ahmad i18-1269
# Mushtaq 18i-1208

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

Saving name_list_train.txt to name_list_train.txt


{'name_list_train.txt': b'giraffe_0\ngiraffe_1\ngiraffe_2\ngiraffe_3\ngiraffe_4\ngiraffe_5\ngiraffe_6\ngiraffe_7\ngiraffe_8\ngiraffe_9\ngiraffe_10\ngiraffe_11\ngiraffe_12\ngiraffe_13\ngiraffe_14\ngiraffe_15\ngiraffe_16\ngiraffe_17\ngiraffe_18\ngiraffe_19\ngiraffe_20\ngiraffe_21\ngiraffe_22\ngiraffe_23\ngiraffe_24\ngiraffe_25\ngiraffe_26\ngiraffe_27\ngiraffe_28\ngiraffe_29\ngiraffe_30\ngiraffe_31\ngiraffe_32\ngiraffe_33\ngiraffe_34\ngiraffe_35\ngiraffe_36\ngiraffe_37\ngiraffe_38\ngiraffe_39\ngiraffe_40\ngiraffe_41\ngiraffe_42\ngiraffe_43\ngiraffe_44\ngiraffe_45\ngiraffe_46\ngiraffe_47\ngiraffe_48\ngiraffe_49\ngiraffe_50\ngiraffe_51\ngiraffe_52\ngiraffe_53\ngiraffe_54\ngiraffe_55\ngiraffe_56\ngiraffe_57\ngiraffe_58\ngiraffe_59\ngiraffe_60\ngiraffe_61\ngiraffe_62\ngiraffe_63\ngiraffe_64\ngiraffe_65\ngiraffe_66\ngiraffe_67\ngiraffe_68\ngiraffe_69\ngiraffe_70\ngiraffe_71\ngiraffe_72\ngiraffe_73\ngiraffe_74\ngiraffe_75\ngiraffe_76\ngiraffe_77\ngiraffe_78\ngiraffe_79\ngiraffe_80\ngiraffe_81\n

In [0]:
!ls

gdrive	name_list_train.txt  sample_data


In [0]:
!unzip /content/gdrive/My\ Drive/FAST-NU/sem-3/NN/Archive.zip


Archive:  /content/gdrive/My Drive/FAST-NU/sem-3/NN/Archive.zip
   creating: bbox_txt_re/
  inflating: bbox_txt_re/giraffe_0.txt  
   creating: __MACOSX/
   creating: __MACOSX/bbox_txt_re/
  inflating: __MACOSX/bbox_txt_re/._giraffe_0.txt  
  inflating: bbox_txt_re/giraffe_1.txt  
  inflating: __MACOSX/bbox_txt_re/._giraffe_1.txt  
  inflating: bbox_txt_re/giraffe_10.txt  
  inflating: bbox_txt_re/giraffe_100.txt  
  inflating: bbox_txt_re/giraffe_1000.txt  
  inflating: bbox_txt_re/giraffe_1001.txt  
  inflating: bbox_txt_re/giraffe_1002.txt  
  inflating: bbox_txt_re/giraffe_1003.txt  
  inflating: bbox_txt_re/giraffe_1004.txt  
  inflating: bbox_txt_re/giraffe_1005.txt  
  inflating: bbox_txt_re/giraffe_1006.txt  
  inflating: bbox_txt_re/giraffe_1007.txt  
  inflating: bbox_txt_re/giraffe_1008.txt  
  inflating: bbox_txt_re/giraffe_1009.txt  
  inflating: bbox_txt_re/giraffe_101.txt  
  inflating: bbox_txt_re/giraffe_1010.txt  
  inflating: bbox_txt_re/giraffe_1011.txt  
  inflatin

In [0]:
!ls dataset


images_re  name_list_train.txt	target_bbox_txt


In [0]:
!mkdir dataset


mkdir: cannot create directory ‘dataset’: File exists


In [0]:
!cp -rf name_list_train.txt dataset

In [0]:
!cp -rf images_re dataset

In [0]:
!cp -rf target_bbox_txt dataset

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 30 23:50:29 2019

@author: ubuntu
"""

import torch
from torch import nn as nn
from torch import optim 
import math
import os
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
import cv2


label_root='./dataset/target_bbox_txt/*.txt'
image_root='./dataset/images_re/*.jpg'
list_root='./dataset/'



class customDataset(Dataset):
    
    def __init__(self, list_file, img_dir, text_dir, transform=None):
        # list of images to load in a .txt file
        self.images = open(list_file, "rt").read().split("\n")[:-1]
        self.transform = transform
        # note that in the .txt file the image names are stored without the extension(.jpg or .txt)
        self.img_extension = ".jpg"
        self.text_extension = ".txt"

        self.image_root_dir = img_dir
        self.text_root_dir = text_dir


    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        name = self.images[index]
        image_path = os.path.join(self.image_root_dir, name + self.img_extension)
        text_path = os.path.join(self.text_root_dir, name + self.text_extension)

        image = self.load_image(path=image_path)
        label = self.load_text(path=text_path)
        #data = {
         #           'image': torch.FloatTensor(image),
          #          'label' : torch.LongTensor(label)
           #         }
        return image, label


    def load_image(self, path=None):
        # can use any other library too like OpenCV as long as you are consistent with it
        raw_image = Image.open(path)
        raw_image = np.transpose(raw_image, (2,1,0))
        imx_t = torch.from_numpy(np.array(raw_image, dtype=np.float32)/255.0)
        print(type(imx_t))
        #imx_t=torch.ones(3,450,450)
        #img=cv2.imread(path)
        #img=torch.tensor(img).transpose(0,2)

        return imx_t
    # can comment the below function if not needed
    def load_text(self, path=None):
        imx_t = torch.from_numpy(np.loadtxt(path))
        print(type(imx_t))
        #raw_image = Image.open(path)
        #imx_t = np.array(raw_image)
        return imx_t
    
        #pwd = '/Volumes/Seagate Expansion Drive/FAST/semester 3/coco/final selected'
        #list_file = pwd+'/input/name_list_train.txt'
        #img_dir = pwd+'/input/images_re'
        #text_dir = pwd+'/input/target_bbox_txt'

        for image, label in ds:
            print(type(label))

mydataset=customDataset('./dataset/name_list_train.txt','./dataset/images_re','./dataset/target_bbox_txt')
trainset=torch.utils.data.DataLoader(mydataset,batch_size=100, shuffle=True)



class Print(nn.Module):
    def forward(self, x):
        print(x.size())
        return x.size()

class myNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        #----Conv1 
        self.conn1=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.pool1=nn.MaxPool2d(2, 2)
        self.actv1=nn.ReLU()
        
        self.conn2=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.pool2=nn.MaxPool2d(2, 2)
        self.actv2=nn.ReLU()
        
        self.conn3=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.pool3=nn.MaxPool2d(2, 2)
        self.actv3=nn.ReLU()
        
        self.conn4=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.pool4=nn.MaxPool2d(2, 2)
        self.actv4=nn.ReLU()
        self.see=Print()
        self.fc1=nn.Linear(32*28*28,125)
        self.actv5=nn.Sigmoid()
#       
    def forward(self,x):
        x=self.actv1(self.pool1(self.conn1(x)))
        x=self.actv2(self.pool2(self.conn2(x)))
        x=self.actv3(self.pool3(self.conn3(x)))
        x=self.actv4(self.pool4(self.conn4(x)))
        temp=self.see(x)
        x=x.view(temp[0],-1)
        x=self.actv5(self.fc1(x))
        return x
        
model = myNet() 



#-------------------The yolo loss function ----------------------------
def indicator(input):
    if input[0]==1:
        return 1.0
    else:
        return 0.0

def yoloLoss(output, target):
    #loss=torch.tensor([1], dtype=torch.float)
    output=output.view(output.shape[0],25,5)
    target=target.view(target.shape[0],25,5)
    ind=0
    loss=0
    
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            ind=indicator(target[i][j])
            loss+=ind*((target[i][j][1]-output[i][j][1])**2+\
                      (target[i][j][2]-output[i][j][2])**2+\
                      (math.sqrt(abs(target[i][j][3]))-math.sqrt(abs(output[i][j][3])))**2+\
                      (math.sqrt(abs(target[i][j][4]))-math.sqrt(abs(output[i][j][4])))**2+
                      (target[i][j][0]-output[i][j][0]))
            
            if ind==0:
                loss+=(target[i][j][0]-output[i][j][0])*.025
                
    loss=loss/output.shape[0]
                
    return loss


images,labels=next(iter(trainset))


optimizer=optim.SGD(model.parameters(),lr=0.003, momentum=0.9)

EPOCHS=2
running_loss=0
for i in range(EPOCHS):
    training_loss=0
    for images, labels in trainset:
        optimizer.zero_grad()
        output=model(images)
        #print(output.view(10,25,5))
        loss=yoloLoss(output,labels)
        
        loss.backward()
        optimizer.step()
    
        print(loss.item())    
    
    running_loss+=loss.item()




<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor

In [0]:
images,label=next(iter(trainset))
#img = images[0]
with torch.no_grad():
  output=model(images)
for out in output:
  print(out)
      

    
    
        

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor